# Getting started with MovingPandas

<img align="right" src="https://anitagraser.github.io/movingpandas/pics/movingpandas.png">

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/anitagraser/movingpandas-examples/main?filepath=1-tutorials/1-getting-started.ipynb)

MovingPandas provides a trajectory datatype based on GeoPandas.
The project home is at https://github.com/anitagraser/movingpandas

The documentation is available at https://movingpandas.readthedocs.io/en/master/

In [ ]:
import pandas as pd
import geopandas as gpd
from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
import movingpandas as mpd

print(mpd.__version__)

## Creating a trajectory from scratch

Trajectory objects consist of a trajectory ID and a GeoPandas GeoDataFrame with a DatetimeIndex. The data frame therefore represents the trajectory data as a Pandas time series with associated point locations (and optional further attributes).

Let's create a small toy trajectory to see how this works:

In [ ]:
df = pd.DataFrame([
  {'geometry':Point(0,0), 't':datetime(2018,1,1,12,0,0)},
  {'geometry':Point(6,0), 't':datetime(2018,1,1,12,6,0)},
  {'geometry':Point(6,6), 't':datetime(2018,1,1,12,10,0)},
  {'geometry':Point(9,9), 't':datetime(2018,1,1,12,15,0)}
]).set_index('t')
geo_df = GeoDataFrame(df, crs=31256)
toy_traj = mpd.Trajectory(geo_df, 1)
toy_traj

In [ ]:
toy_traj.plot()

We can also access the trajectory's GeoDataFrame:

In [ ]:
toy_traj.df

In [ ]:
toy_traj.df.plot()

## Loading trajectory data from a GeoPackage

The MovingPandas repository contains a demo GeoPackage file that can be loaded as follows:

In [ ]:
%%time
gdf = read_file('data/demodata_geolife.gpkg')
gdf['t'] = pd.to_datetime(gdf['t'])
gdf = gdf.set_index('t').tz_localize(None)
print("Finished reading {} rows".format(len(df)))

After reading the trajectory point data from file, we want to construct the trajectories.

### Creating trajectories with TrajectoryCollection

TrajectoryCollection is a convenience class that takes care of creating trajectories from a GeoDataFrame:

In [ ]:
traj_collection = mpd.TrajectoryCollection(gdf, 'trajectory_id')
print(traj_collection)

In [ ]:
traj_collection.plot(column='trajectory_id', legend=True, figsize=(9,5))

### Accessing individual trajectories

In [ ]:
my_traj = traj_collection.trajectories[1]
print(my_traj)

In [ ]:
my_traj.plot(linewidth=5, capstyle='round', figsize=(9,3))

To visualize trajectories in their geographical context, we can also create interactive plots with basemaps:

In [ ]:
my_traj.hvplot(width=500, height=300, line_width=7.0, tiles='StamenTonerBackground')